In [1]:
import yaml
import random
import numpy as np
import time

from reader import Reader

from const import Const
from utils import get_F_example
from clustering import Clusters
from i_merge import IMerger
from Subclusterring import Subclusters

from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd

# Если есть желание создать свои данные, то вызвать этот код

In [2]:
# X, y = make_blobs(n_samples=200, centers=7, cluster_std=0.6, random_state=0)

# ids = [i+1 for i in range(len(X))]
# df = pd.DataFrame()
# df['id'] = ids
# df['X1'] = X[:,0]
# df['X2'] = X[:,1]

In [3]:
# ids = []
# X1 = []
# X2 = []
# with open('Two.txt', 'r') as f:
#     for k, line in enumerate(f):
#         if k==0:
#             continue
#         ids.append(int(line.split()[0]))
#         X1.append(float(line.split()[1]))
#         X2.append(float(line.split()[2]))
#         if k>=205:
#             break
# df = pd.DataFrame()
# df['id'] = ids
# df['X1'] = X1
# df['X2'] = X2
    

In [4]:
reader = Reader()

In [5]:
df = reader.read('./Data/3.xlsx')

Stated size of data 693
Data contained  1 incorrect rows
Size of data 692
X1 contain 1122 zero distances 0.00234
Interval: 0.03519933350192
Interval: 0.11539371202450582
Interval: 0.20149512078691345
Interval: 0.2969515410767534
Interval: 0.40178160276492036
Interval: 0.5127979223990603
Interval: 0.6273898157708088
Interval: 0.7595600809004875
Interval: 0.9493474555820113
Interval: 1.7146341818751325
histogram [334046 126990  12318    480    642    260    186   1758   1650    534]
bins lenght [0.        0.6863083 1.3726166 2.0589249 2.7452332 3.4315415 4.1178498
 4.8041581 5.4904664 6.1767747 6.863083 ]
X2 contain 1062 zero distances 0.00222
Interval: 0.09030509518384339
Interval: 0.3017736365534963
Interval: 0.5304085558200842
Interval: 0.8011414554149474
Interval: 1.0777258516476662
Interval: 1.3946911772543165
Interval: 1.7700571379944048
Interval: 2.160520571106381
Interval: 2.5873532292110446
Interval: 3.4000250034662782
histogram [136658 104100  80580  75022  48246  25196   6510 

In [6]:
df, df_test = reader.split_data(df, train_size=20)

In [ ]:
df.shape

In [ ]:
plt.scatter(df['X1'], df['X2'])
plt.show()

# Подбор констант

In [ ]:
const = Const()

In [ ]:
const.norm(df) # Нормировка

In [ ]:
const.get_norms() # Получение норм

In [ ]:
a = time.time()
const.calculate_a(df, type_of_optimization=4) # Вычисление а и пересчет констант
b = time.time()
print(b-a)

In [ ]:
const.add_Fcolumn(df)

In [ ]:
const.config # Вывод параметров

In [ ]:
const.save_consts('example1') #Сохранение констант в файл

Researching dataset

In [ ]:
# fig = px.scatter_3d(df, x='X1', y='X2', z='F')
# fig.show()

# Кластеризация

In [ ]:
cluster = Clusters(const.config) 

In [ ]:
df = cluster.get_isolated_clusters(df) # Вычисление кластеров
df.head()

In [ ]:
const.config

In [ ]:
set(df['cluster_id'])

In [ ]:
for cluster in set(df['cluster_id']):
    plt.scatter(df[df['cluster_id']==cluster]['X1'], df[df['cluster_id']==cluster]['X2'])


In [ ]:
fig = px.scatter_3d(df, x='X1', y='X2', z='X3',
              color='cluster_id')
fig.show()

In [ ]:
df.head()

# Merge I

In [ ]:
df

In [ ]:
Merger = IMerger(const.config)
Merger.mergeClusters(df)

In [ ]:
plt.scatter(df['X1'],df['X2'], c=df['cluster_id'])

In [ ]:
fig = px.scatter_3d(df, x='X1', y='X2', z='X3',
              color='cluster_id')
fig.show()

# Subcluster

In [ ]:
sub = Subclusters(const.config)

In [ ]:
df = sub.subclustering(df, type_of_closed=2)

In [ ]:
fig = px.scatter(df, x="X1", y="X2", color="subcluster_id", hover_data=['id', 'F'], )
fig.show()

In [ ]:
fig = px.scatter_3d(df, x='X1', y='X2', z='X3',
              color='subcluster_id')
fig.show()

# Last Stage

In [ ]:
df.head()

In [ ]:
class Classsifier():
    def __init__(self, mean, config):
        self.mean = mean
        self.config = config
        self.ignorename = ['cluster_id', 'subcluster_id']
        
    def __norm(self, df, mean):
        for name in self.mean:
            df[name] /= mean[name]
        return df
    
    def predict(self, df_train, df_predict, a=None):
        names_ouput = list(df_train)[1:]#Выделяем все имена из обучающих данных  
        names_input = list(df_predict)[1:]#Выделяем все имена из данных для предсказания
        needed_mean = {}
        needed_names = ['id']
        for name in names_input:
            if name in names_ouput:
                needed_names.append(name)
                needed_mean[name] = self.mean[name]
        if len(needed_names)==1:
            print('Нет общих координат') #Если нет общих координат, то прерываем программу
            return None
        
        df_train = df_train[needed_names+self.ignorename] # Выделяем необходимые имена
        df_predict = df_predict[needed_names] # Выделяем необходимые имена
        df_predict = self.__norm(df_predict, self.mean) #Нормируем данные
        
        inputs = df_predict.iloc[:].values
        
        for cluster in sorted(df_train['cluster_id'].unique()): #проходим все кластеры
            time_df = df_train[lambda x: x['cluster_id']==cluster] #Выделяем все строки относящиеся к этому кластеру
            for subcluster in sorted(time_df['subcluster_id'].unique()): #проходим все подкластеры
                name = 'F_'+str(cluster) +'_' + str(subcluster) # Формируем имя столбца
                time_dfs = time_df[lambda x: x['subcluster_id']==subcluster] #Выделяем все строки относящиеся к этому подкластеру
                F = []
                for line in inputs:
                    F.append(get_F_example(time_dfs[needed_names].iloc[:].values, self.config['consts']['a'], line)) #Вычисляем F для каждой строки данных для предсказания
                df_predict[name] = F
        return df_predict

In [ ]:
clas = Classsifier(const.get_norms(), const.config)

In [ ]:
df_predict = clas.predict(df, df_test)

In [ ]:
df.head()

In [ ]:
ignorename = ['F_0_0','F_0_1','F_0_2','F_0_3','F_0_4','F_0_5','F_0_6']
df_predict['cluster_id'] = df_predict[ignorename].idxmax(axis=1)
df_predict.head()

In [ ]:
fig = px.scatter_3d(df_predict, x='X1', y='X2', z='X3',
              color='cluster_id')
fig.show()